In [ ]:
from imports import *

mySuperSim = "/scratch/user/kelsey/software/supersim/CDMSpython"
sys.path.insert(0, mySuperSim)
import dmc

In [ ]:
help(dmc.QETLayout)

In [ ]:
# QET drawing functions, adapted from Rik Bhattacharyya

import os
from matplotlib.patches import Rectangle

def loadQETmask(det, side="top"):
    """Load QET mask data into Pandas dataframe from SuperSim mask file.
       'det' is detector type (iZIP5, HV100mm, etc.), 'side' is 'top' or
       'bottom'.  Mask positions are in mm, rotation angle in radians."""
    
    file = os.environ["CDMS_SUPERSIM"]+f"/CDMSgeometry/data/dmc/{det}/{side}_mask.dat"
    mask = pd.read_csv(file,names=['x','y','phirot'],header=None,sep=r"\s+")
    return mask

def drawQETpositions(mask, ax=None, **kwargs):
    """Show positions of QETs with dots at centers, using Pandas 'mask' data read
       from SuperSim mask file.  User may pass in desired matplotlib figure as 'ax',
       or a new figure will be created.  Any arguments to plt.scatter() can be passed
       by keyword."""
    
    if ax is None: fig, ax = plt.subplots(figsize=(6,6))
    ax.scatter(mask['x'], mask['y'], **kwargs)
    return ax
    
def drawQETboxes(mask, ax=None, length_um=672.6, width_um=240.0, label=None, **kwargs):
    """Draw QET boxes from Numpy or Pandas 'mask' data read from SuperSim mask file.
       User may pass in desired matplotlib figure as 'ax', or a new figure will be created.
       QET dimensions should be passed in microns; will be converted to mm internally.
       The optional 'label' argument will only be used with a single QET box.  Any
       other arguments to matplotlib Rectangle() can be passed by keyword."""
    
    if ax is None: fig, ax = plt.subplots(figsize=(6,6))

    # convert microns to mm
    L = length_um * 1e-3
    W = width_um  * 1e-3

    phirot = np.degrees(mask['phirot'])     # Convert tangent angle to degrees

    for i, (cx, cy, angle) in enumerate(zip(mask['x'], mask['y'], phirot)):
        # Axis-aligned rectangle centered at (cx, cy)
        rect = Rectangle((cx-W/2, cy-L/2), W, L,
                         angle=angle, rotation_point='center',
                         label=label if (i == 0 and label is not None) else None,
                         **kwargs)
        ax.add_patch(rect)

    return ax

In [ ]:
# Demonstrate QET drawing
mask = loadQETmask("iZIP5","top")
fig,ax = plt.subplots(figsize=(8,8))

drawQETpositions(mask, ax, c='red', s=10, alpha=0.6, label="QET centers")

# Draw QET boxes
drawQETboxes(mask, ax=ax, edgecolor='green', facecolor='none', lw=1.0, alpha=0.9, label="QETs")

# Zoomed region
ax.set_xlim(20, 38)
ax.set_ylim(-10, 10)

# Equal aspect
ax.set_aspect("equal", adjustable="box")

# Legend styling
ax.legend(loc=3, fontsize=14, markerscale=2, frameon=True, fancybox=False, framealpha=0.8)

# Axis labels + title
ax.set_xlabel("X position [mm]", fontsize=16)
ax.set_ylabel("Y position [mm]", fontsize=16)
ax.set_title("QET Mask Layout \n(iZIP5 Top Mask)", fontsize=18, pad=15)

plt.show()

In [ ]:
# Demonstrate QET drawing using new SuperSim class

fig,ax = plt.subplots(figsize=(8,8))

qetLayout = dmc.QETLayout("iZIP5","top")
qetLayout.DrawPositions(ax, label="QET centers")
qetLayout.DrawBoxes(ax, edgecolor='green', facecolor='none', lw=1.0, alpha=0.9, label="QETs")

# Zoomed region
ax.set_xlim(20, 38)
ax.set_ylim(-10, 10)

# Equal aspect
ax.set_aspect("equal", adjustable="box")

# Legend styling
ax.legend(loc=3, fontsize=14, markerscale=2, frameon=True, fancybox=False, framealpha=0.8)

# Axis labels + title
ax.set_xlabel("X position [mm]", fontsize=16)
ax.set_ylabel("Y position [mm]", fontsize=16)
ax.set_title("QET Mask Layout \n(iZIP5 Top Mask)", fontsize=18, pad=15)

plt.show()